In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter 
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain


In [4]:
loader = Docx2txtLoader("/Users/rouzbeh/Downloads/6.19.2023 SamplePatientNotes_Compiled.docx")
documents = loader.load()
# split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
texts = text_splitter.split_documents(documents)
# select which embeddings we want to use
embeddings = OpenAIEmbeddings()
# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":4})
# create a chain to answer questions 
qa = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(model_name="gpt-4", temperature=0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    )
chat_history = []
query = "Can you give a short summary of this document?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]


Created a chunk of size 1578, which is longer than the specified 1000
Created a chunk of size 1699, which is longer than the specified 1000
Created a chunk of size 1240, which is longer than the specified 1000
Created a chunk of size 1214, which is longer than the specified 1000
Created a chunk of size 1501, which is longer than the specified 1000
Created a chunk of size 1238, which is longer than the specified 1000
Created a chunk of size 1147, which is longer than the specified 1000
Created a chunk of size 1942, which is longer than the specified 1000
Created a chunk of size 1269, which is longer than the specified 1000
Created a chunk of size 1082, which is longer than the specified 1000
Created a chunk of size 1460, which is longer than the specified 1000


"This document is a collection of medical records for a male patient, aged 42-44, who has a history of cystic fibrosis and has undergone a lung transplant. The patient has also been treated for a bee sting that resulted in cellulitis. The patient has been seen by several physicians and healthcare professionals, including those specializing in pulmonary disease and family practice. The patient's post-transplant follow-ups have been generally positive, with occasional nausea reported but no other significant issues. The patient has also received immunizations without any initial adverse effects."

In [5]:
chat_history = [(query, result["answer"])]
query = "Is there anything the patient is allergic to?"
result = qa({"question": query, "chat_history": chat_history})
result['answer']

'Yes, the patient has allergies to Vitamin K, Intravenous Vitamin K, and Bee Venom. The reactions to these allergens are Anaphylaxis for Vitamin K, and Fever and Swelling for Bee Venom. The patient also has an allergy to NSAIDs due to a lung transplant.'

In [6]:
chat_history = [(query, result["answer"])]
query = "Is there any pills patient currently using?"
result = qa({"question": query, "chat_history": chat_history})
result['answer']

'The patient is currently taking the following medications:\n\n1. Aspirin 81 MG Oral Tab - 81 mg daily\n2. Cetirizine HCl (ZYRTEC OR) - 10 mg daily\n3. Cholecalciferol 1000 UNITS Oral Tab - 1,000 Units daily\n4. Cyanocobalamin (VITAMIN B 12 OR)\n5. DilTIAZem HCl ER Beads 360 MG Oral CAPSULE SR 24 HR - 1 capsule daily\n6. EPINEPHrine 0.3 MG/0.3ML Injection Solution Auto-injector - 0.3 mL (0.3 mg) intramuscularly one time as needed for anaphylaxis\n7. FREESTYLE LANCETS Misc - Use 1 each 6 times a day\n8. Glucose Blood (FREESTYLE LITE TEST) In Vitro Strip - 1 strip by In Vitro route 6 times a day\n9. AMLODIPINE 5 MG Oral Tab - 1 tablet daily\n\nAdditionally, the patient is also taking the following medications as reviewed with the patient:\n\n1. Acetaminophen 650mg q6 PRN\n2. ASA 81mg qd\n3. Calcium carbonate 1000mg bid\n4. Cholecalciferol 3000IU qd\n5. Docusate 250mg bid\n6. Fluconazole 400mg qd\n7. Gabapentin 300mg tid\n8. Glargine 8u qhs\n9. NPH 8u q12\n10. Humalog 6/6/6 +SSI\n11. Lora

In [7]:
chat_history = [(query, result["answer"])]
query = "Should the patient worry about anything in particular?"
result = qa({"question": query, "chat_history": chat_history})
result['answer']

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).


"As an AI, I can't make medical judgments, but I can tell you that the patient is taking Aspirin, which is a blood thinning medication. According to the context provided, patients on blood thinning medications should advise their providers as specific precautions need to be taken when giving intramuscular vaccines. They should also monitor for signs and symptoms of hematoma formation at the site of the injection. It's always best for the patient to discuss their medications and any potential concerns with their healthcare provider."